In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sepsis/Paitients_Files_Train.csv
/kaggle/input/sepsis/Paitients_Files_Test.csv
/kaggle/input/sepsis/README.md


# Although we have a small dataset which is not enough for artificial neural networks(ANN), we have used ANN to predict sepsis, with a larger dataset we can use this to predict Sepsis with much higher accuracy.

In [2]:
import os    
from keras.models import Sequential
from keras.layers import Dense

In [3]:
df = pd.read_csv("/kaggle/input/sepsis/Paitients_Files_Train.csv")

df

,ID,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance,Sepssis
0,ICU200010,6,148,72,35,0,33.6,0.627,50,0,Positive
1,ICU200011,1,85,66,29,0,26.6,0.351,31,0,Negative
2,ICU200012,8,183,64,0,0,23.3,0.672,32,1,Positive
3,ICU200013,1,89,66,23,94,28.1,0.167,21,1,Negative
4,ICU200014,0,137,40,35,168,43.1,2.288,33,1,Positive
...,...,...,...,...,...,...,...,...,...,...,...
594,ICU200604,6,123,72,45,230,33.6,0.733,34,0,Negative
595,ICU200605,0,188,82,14,185,32.0,0.682,22,1,Positive
596,ICU200606,0,67,76,0,0,45.3,0.194,46,1,Negative
597,ICU200607,1,89,24,19,25,27.8,0.559,21,0,Negative


In [4]:
# encoding categorical
df_one = pd.get_dummies(df["Sepssis"])
df_two = pd.concat((df_one, df), axis=1)
df_two = df_two.drop(["Sepssis"], axis=1)
df_two = df_two.drop(["Negative"], axis=1)
df = df_two.rename(columns={"Positive": "Sepssis"})
print(df)

     Sepssis         ID  PRG   PL  PR  SK   TS   M11    BD2  Age  Insurance
0          1  ICU200010    6  148  72  35    0  33.6  0.627   50          0
1          0  ICU200011    1   85  66  29    0  26.6  0.351   31          0
2          1  ICU200012    8  183  64   0    0  23.3  0.672   32          1
3          0  ICU200013    1   89  66  23   94  28.1  0.167   21          1
4          1  ICU200014    0  137  40  35  168  43.1  2.288   33          1
..       ...        ...  ...  ...  ..  ..  ...   ...    ...  ...        ...
594        0  ICU200604    6  123  72  45  230  33.6  0.733   34          0
595        1  ICU200605    0  188  82  14  185  32.0  0.682   22          1
596        0  ICU200606    0   67  76   0    0  45.3  0.194   46          1
597        0  ICU200607    1   89  24  19   25  27.8  0.559   21          0
598        1  ICU200608    1  173  74   0    0  36.8  0.088   38          1

[599 rows x 11 columns]


In [5]:
# Separate Target Variable and Predictor Variables
TargetVariable=['Sepssis']
Predictors=['PRG', 'PL', 'PR', 'SK', 'TS', 'M11',
            'BD2', 'Age', 'Insurance']

X=df[Predictors].values
y=df[TargetVariable].values


### Sandardization of data ###
### We does not standardize the Target variable for classification
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Quick sanity check with the shapes of Training and Testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(419, 9)
(419, 1)
(180, 9)
(180, 1)


In [6]:
classifier = Sequential()
# Defining the Input layer and FIRST hidden layer,both are same!
# relu means Rectifier linear unit function
classifier.add(Dense(units=10, input_dim=9, kernel_initializer='uniform', activation='relu'))

#Defining the SECOND hidden layer, here we have not defined input because it is
# second layer and it will get input as the output of first hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

# Defining the Output layer
# sigmoid means sigmoid activation function
# for Multiclass classification the activation ='softmax'
# And output_dim will be equal to the number of factor levels
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Optimizer== the algorithm of SGG to keep updating weights
# loss== the loss function to measure the accuracy
# metrics== the way we will compare the accuracy after each step of SGD
classifier.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# fitting the Neural Network on the training data
survivalANN_Model=classifier.fit(X_train,y_train, batch_size=42 , epochs=20, verbose=1)



Epoch 1/20
10/10 [==============================] - 1s 3ms/step - loss: 0.6915 - accuracy: 0.6444
Epoch 2/20
10/10 [==============================] - 0s 3ms/step - loss: 0.6892 - accuracy: 0.6611
Epoch 3/20
10/10 [==============================] - 0s 3ms/step - loss: 0.6871 - accuracy: 0.6611
Epoch 4/20
10/10 [==============================] - 0s 3ms/step - loss: 0.6847 - accuracy: 0.6611
Epoch 5/20
10/10 [==============================] - 0s 3ms/step - loss: 0.6817 - accuracy: 0.6611
Epoch 6/20
10/10 [==============================] - 0s 3ms/step - loss: 0.6779 - accuracy: 0.6611
Epoch 7/20
10/10 [==============================] - 0s 3ms/step - loss: 0.6732 - accuracy: 0.6611
Epoch 8/20
10/10 [==============================] - 0s 3ms/step - loss: 0.6674 - accuracy: 0.6611
Epoch 9/20
10/10 [==============================] - 0s 3ms/step - loss: 0.6606 - accuracy: 0.6611
Epoch 10/20
10/10 [==============================] - 0s 3ms/step - loss: 0.6526 - accuracy: 0.6611
Epoch 11/20
10/10 [

In [7]:

# Predictions on testing data
Predictions=classifier.predict(X_test)
 
# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)
 
# Generating a data frame for analyzing the test data
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['Sepsis']=y_test
TestingData['PredictedSepsisProb']=Predictions
 
# Defining the probability threshold
def probThreshold(inpProb):
    if inpProb  >  0.5:
        return(1)
    else:
        return(0)
 
# Generating predictions on the testing data by applying probability threshold
TestingData['PredictedSepsis']=TestingData['PredictedSepsisProb'].apply(probThreshold)
print(TestingData.head())
 
###############################################
from sklearn import metrics
print('\n######### Testing Accuracy Results #########')
print(metrics.classification_report(TestingData['Sepsis'], TestingData['PredictedSepsis']))
print(metrics.confusion_matrix(TestingData['Sepsis'], TestingData['PredictedSepsis']))

6/6 [==============================] - 0s 2ms/step
   PRG     PL    PR    SK     TS   M11    BD2   Age  Insurance  Sepsis  \
0  3.0  171.0  72.0  33.0  135.0  33.3  0.199  24.0        1.0       1   
1  3.0  129.0  64.0  29.0  115.0  26.4  0.219  28.0        1.0       1   
2  0.0   91.0  80.0   0.0    0.0  32.4  0.601  27.0        0.0       0   
3  5.0   95.0  72.0  33.0    0.0  37.7  0.370  27.0        1.0       0   
4  0.0  119.0  64.0  18.0   92.0  34.9  0.725  23.0        0.0       0   

   PredictedSepsisProb  PredictedSepsis  
0             0.418323                0  
1             0.285546                0  
2             0.200829                0  
3             0.328259                0  
4             0.277748                0  

######### Testing Accuracy Results #########
              precision    recall  f1-score   support

           0       0.63      1.00      0.78       114
           1       0.00      0.00      0.00        66

    accuracy                           0.6

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Bonus code to find out he best hyperparameters 

In [8]:
# Function to generate Deep ANN model 
def make_classification_ann(Optimizer_Trial, Neurons_Trial):
    from keras.models import Sequential
    from keras.layers import Dense
    
    # Creating the classifier ANN model
    classifier = Sequential()
    classifier.add(Dense(units=Neurons_Trial, input_dim=9, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=Neurons_Trial, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(optimizer=Optimizer_Trial, loss='binary_crossentropy', metrics=['accuracy'])
            
    return classifier

########################################

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


Parameter_Trials={'batch_size':[10,20,30],
                      'epochs':[10,20],
                    'Optimizer_Trial':['adam', 'rmsprop'],
                  'Neurons_Trial': [5,10]
                 }

# Creating the classifier ANN
classifierModel=KerasClassifier(make_classification_ann, verbose=0)

########################################

# Creating the Grid search space
# See different scoring methods by using sklearn.metrics.SCORERS.keys()
grid_search=GridSearchCV(estimator=classifierModel, param_grid=Parameter_Trials, scoring='f1', cv=5)

########################################

# Measuring how much time it took to find the best params
import time
StartTime=time.time()

# Running Grid Search for different paramenters
grid_search.fit(X_train,y_train, verbose=1)

EndTime=time.time()
print("############### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes #############')

########################################

# printing the best parameters
print('\n#### Best hyperparamters ####')
grid_search.best_params_

# Training the model with best hyperparamters
classifier.fit(X_train,y_train, batch_size=10 , epochs=20, verbose=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/10
34/34 [==============================] - 1s 2ms/step - loss: 0.6910 - accuracy: 0.6627
Epoch 2/10
34/34 [==============================] - 0s 2ms/step - loss: 0.6863 - accuracy: 0.6657
Epoch 3/10
34/34 [==============================] - 0s 2ms/step - loss: 0.6802 - accuracy: 0.6657
Epoch 4/10
34/34 [==============================] - 0s 3ms/step - loss: 0.6696 - accuracy: 0.6657
Epoch 5/10
34/34 [==============================] - 0s 2ms/step - loss: 0.6515 - accuracy: 0.6657
Epoch 6/10
34/34 [==============================] - 0s 2ms/step - loss: 0.6208 - accuracy: 0.6657
Epoch 7/10
34/34 [==============================] - 0s 2ms/step - loss: 0.5791 - accuracy: 0.6657
Epoch 8/10
34/34 [==============================] - 0s 2ms/step - loss: 0.5403 - accuracy: 0.6657
Epoch 9/10
34/34 [==============================] - 0s 2ms/step - loss: 0.5091 - accuracy: 0.7045
Epoch 10/10
3/3 [==============================] - 0s 3ms/step
Epoch 1/10
34/34 [==============================] - 1s 